In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('podcasts_entries.csv', header=None,error_bad_lines=False)
    
rt_cnt=len(set(df[1])) 
us_cnt=len(set(df[3]))
users= dict()
movies= dict()
print us_cnt
    
utility=np.zeros((us_cnt,rt_cnt))
us_cnt_tr=0
mv_cnt_tr=0
for i in range(0, len(df)):
    if df[3][i] not in users:
        users[df[3][i]]= us_cnt_tr
        us_cnt_tr=us_cnt_tr+1
    if df[1][i] not in movies:
        movies[df[1][i]]= mv_cnt_tr
        mv_cnt_tr=mv_cnt_tr+1
    idx_us= users[df[3][i]]
    idx_mv= movies[df[1][i]]
    utility[idx_us][idx_mv]=1.0
    #print utility[0]
s=np.sum(utility, axis=1, keepdims=True)
count=0
for i in range(0, len(s)):
    if s[i] > 1.0:
        count=count+1

us_cnt_tr=0
utility_nw=np.zeros((count,rt_cnt))
for i in range(0, len(utility)):
    if s[i] > 1:
        for j in range(0, len(utility[i])):
            utility_nw[us_cnt_tr][j]= utility[i][j]
        us_cnt_tr=us_cnt_tr+1    
    #print count
print len(utility_nw)
fp= open("users.txt","w")
fp.writelines(str(users))
fp.close()
    
fp= open("ratings.txt","w")
fp.writelines(str(movies))
fp.close()
    
np.savetxt('test.txt', s, delimiter=',')
np.savetxt('utl_nw.txt', utility_nw, delimiter=',')

Skipping line 74: expected 4 fields, saw 5
Skipping line 300: expected 4 fields, saw 6
Skipping line 634: expected 4 fields, saw 5
Skipping line 766: expected 4 fields, saw 5
Skipping line 895: expected 4 fields, saw 5
Skipping line 1377: expected 4 fields, saw 5
Skipping line 1558: expected 4 fields, saw 5
Skipping line 1752: expected 4 fields, saw 5
Skipping line 1959: expected 4 fields, saw 5
Skipping line 2031: expected 4 fields, saw 5
Skipping line 2099: expected 4 fields, saw 5
Skipping line 2156: expected 4 fields, saw 5
Skipping line 2195: expected 4 fields, saw 5
Skipping line 2257: expected 4 fields, saw 5
Skipping line 2259: expected 4 fields, saw 5
Skipping line 2293: expected 4 fields, saw 5
Skipping line 2394: expected 4 fields, saw 5
Skipping line 2407: expected 4 fields, saw 5
Skipping line 2540: expected 4 fields, saw 5
Skipping line 2545: expected 4 fields, saw 5
Skipping line 2942: expected 4 fields, saw 5
Skipping line 2977: expected 4 fields, saw 5
Skipping line 31

8344
992


In [8]:
from sklearn.metrics import jaccard_similarity_score

sim_mat= np.zeros((count,count))
for i in range(0,count):
    for j in range(0, count):
        sim_mat[i][j]= jaccard_similarity_score(utility_nw[i], utility_nw[j])

In [19]:
idx_mv=5
ind = np.argpartition(sim_mat[idx_mv], -50)[-50:]

best_rated=np.zeros((rt_cnt))
for i in ind:
    for j in range(0, rt_cnt):
        if i != idx_mv:
            best_rated[j]=  best_rated[j]+ utility_nw[i][j]
ind_best = np.argpartition(best_rated, -10)[-10:]
print ind_best

for i in ind_best:
    print utility_nw[idx_mv][i]

[640 126 335 130 477 818 475 697  63 621]
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
